##Question 1

### Download the IMDB Dataset

In [1]:
# Download reviews.txt and labels.txt from here: https://github.com/udacity/deep-learning/tree/master/sentiment-network

def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('/content/drive/My Drive/NLP/Data/reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('/content/drive/My Drive/NLP/Data/labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

### Capturing Word Correlation in Input Data

In [2]:
import numpy as np
import torch

onehots = {}
onehots['cat'] = np.array([1,0,0,0])
onehots['the'] = np.array([0,1,0,0])
onehots['dog'] = np.array([0,0,1,0])
onehots['sat'] = np.array([0,0,0,1])

sentence = ['the','cat','sat']
x = onehots[sentence[0]] + \
    onehots[sentence[1]] + \
    onehots[sentence[2]]

print("Sent Encoding:" + str(x))

Sent Encoding:[1 1 0 1]


### Predicting Movie Reviews

In [3]:
import sys

f = open('/content/drive/My Drive/NLP/Data/reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('/content/drive/My Drive/NLP/Data/labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

In [4]:
## tokens is a list of sets 

In [5]:
vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

In [6]:
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

In [7]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [9]:
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

In [10]:
target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [11]:
# upload model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [12]:
import numpy as np
import tensorflow as tf
from numpy import tensordot
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + torch.exp(-x))

alpha, iterations = (0.00008, 3)
hidden_size = 50

weights_0_1 = 0.2*torch.randn((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*torch.randn((hidden_size,1)) - 0.1

correct,total = (0,0)
for iter in range(iterations):
    
    # train on first 21,000
    for i in range(len(input_dataset)-4000):
     
        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(torch.sum(weights_0_1[x],axis=0)) #embed + sigmoid
        layer_2 = sigmoid(torch.matmul(layer_1,weights_1_2))  # linear + softmax

        layer_2_delta = layer_2 - y # compare pred with truth
        layer_1_delta = sigmoid(torch.matmul(layer_2_delta,weights_1_2.T)) #backprop

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= torch.ger(layer_1,layer_2_delta) * alpha

        if(torch.abs(layer_2_delta) < 0.5000001):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
correct,total = (0,0)
for i in range(len(input_dataset)-4000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(torch.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(torch.matmul(layer_1,weights_1_2))
    
    if(torch.abs(layer_2 - y) < 0.5000001):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))

Iter:0 Progress:83.99% Training Accuracy:0.8424285714285714%
Iter:1 Progress:83.99% Training Accuracy:0.9191904761904762%
Iter:2 Progress:83.99% Training Accuracy:0.946031746031746%
Test Accuracy:0.99975


##Question 2

In [13]:
import torch
n_input, n_hidden, n_output = 2, 2, 1

###Parameter initialization

In [14]:
## initialize tensor for inputs, and outputs 
inp = [1,2]
inp = torch.FloatTensor(inp)
oup = torch.randn((1, n_output))

In [15]:
w1 = ([[0, 0],
      [0, 0]])
w1= torch.FloatTensor(w1)

In [16]:
w2 = ([[0],
       [0]])
w2= torch.FloatTensor(w2)

###Forward Propagation

In [17]:
## sigmoid activation function using pytorch
def sigmoid_activation(z):
    return 1 / (1 + torch.exp(-z))

## activation of hidden layer 
z1 = torch.matmul(inp, w1) 
a1 = sigmoid_activation(z1)

## output of final layer 
# we are not considering activation function of output layer
output = torch.matmul(a1, w2) 

###Loss Computation

###Taking 'Mean Square Error (MSE)' as loss function

In [18]:
l = (oup - output)**2
loss = l/2
loss

tensor([[0.2008]])

###Backpropagation

In [19]:
## function to calculate the derivative of activation
def sigmoid_delta(m):
  return m * (1 - m)

## compute derivative of error terms
delta_hidden = sigmoid_delta(a1)

## backpass the changes to previous layers 
loss_h = torch.matmul(loss, w2.T)
d_hidn = loss_h * delta_hidden

In [20]:
inp_t=inp.view(-1,1)
inp_t

tensor([[1.],
        [2.]])

In [21]:
a1_t=a1.view(-1,1)
a1_t

tensor([[0.5000],
        [0.5000]])

###Updating the Parameters

In [22]:
learning_rate = 0.1

w2 -= torch.matmul(a1_t, loss) * learning_rate
w1 -=  torch.matmul(inp_t,d_hidn) * learning_rate

print ("epoch 1 :")
print ("Output :" + str(output))
print ("Predicted output :" + str(oup))
print ("Loss :"  + str(loss))
print ("W1 :" + str(w1))
print ("W2 :" + str(w2))

epoch 1 :
Output :tensor([0.])
Predicted output :tensor([[-0.6337]])
Loss :tensor([[0.2008]])
W1 :tensor([[0., 0.],
        [0., 0.]])
W2 :tensor([[-0.0100],
        [-0.0100]])


In [23]:
## sigmoid activation function using pytorch
def sigmoid_activation(z):
    return 1 / (1 + torch.exp(-z))

 ## function to calculate the derivative of activation
def sigmoid_delta(m):
  return m * (1 - m)

## activation of hidden layer 
z1 = torch.matmul(inp, w1) 
a1 = sigmoid_activation(z1)

## output of final layer 
# we are not considering activation function of output layer
output = torch.matmul(a1, w2)

l = (oup - output)**2
loss = l/2
loss

## compute derivative of error terms
delta_hidden = sigmoid_delta(a1)

## backpass the changes to previous layers 
loss_h = torch.matmul(loss, w2.T)
d_hidn = loss_h * delta_hidden

learning_rate = 0.1

w2 -= torch.matmul(a1_t, loss) * learning_rate
w1 -=  torch.matmul(inp_t,d_hidn) * learning_rate

print ("epoch 2 :")
print ("Output :" + str(output))
print ("Predicted output :" + str(oup))
print ("Loss :"  + str(loss))
print ("W1 :" + str(w1))
print ("W2 :" + str(w2))

epoch 2 :
Output :tensor([-0.0100])
Predicted output :tensor([[-0.6337]])
Loss :tensor([[0.1945]])
W1 :tensor([[4.8807e-05, 4.8807e-05],
        [9.7614e-05, 9.7614e-05]])
W2 :tensor([[-0.0198],
        [-0.0198]])


In [24]:
## sigmoid activation function using pytorch
def sigmoid_activation(z):
    return 1 / (1 + torch.exp(-z))

 ## function to calculate the derivative of activation
def sigmoid_delta(m):
  return m * (1 - m)

## activation of hidden layer 
z1 = torch.matmul(inp, w1) 
a1 = sigmoid_activation(z1)

## output of final layer 
# we are not considering activation function of output layer
output = torch.matmul(a1, w2)

l = (oup - output)**2
loss = l/2
loss

## compute derivative of error terms
delta_hidden = sigmoid_delta(a1)

## backpass the changes to previous layers 
loss_h = torch.matmul(loss, w2.T)
d_hidn = loss_h * delta_hidden

learning_rate = 0.1

w2 -= torch.matmul(a1_t, loss) * learning_rate
w1 -=  torch.matmul(inp_t,d_hidn) * learning_rate

print ("epoch 3 :")
print ("Output :" + str(output))
print ("Predicted output :" + str(oup))
print ("Loss :"  + str(loss))
print ("W1 :" + str(w1))
print ("W2 :" + str(w2))

epoch 3 :
Output :tensor([-0.0198])
Predicted output :tensor([[-0.6337]])
Loss :tensor([[0.1885]])
W1 :tensor([[0.0001, 0.0001],
        [0.0003, 0.0003]])
W2 :tensor([[-0.0292],
        [-0.0292]])
